# **Worksapce set up**

In [9]:
# Connecting to google drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [10]:
# Import libraries
%pylab inline
!pip install -q tensorflow-model-optimization

import tensorflow as tf
import tensorflow_model_optimization as tfmot
import numpy as np
import tempfile
import os
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

Populating the interactive namespace from numpy and matplotlib


In [11]:
# Display python and library versions
!python --versions
print('Numpy ' + np.__version__)
print('TensorFlow ' + tf.__version__)
print('Keras ' + tf.keras.__version__)

unknown option --versions
usage: python3 [option] ... [-c cmd | -m mod | file | -] [arg] ...
Try `python -h' for more information.
Numpy 1.21.6
TensorFlow 2.8.2
Keras 2.8.0


# **Dataset Management**

In [12]:
# Loads the data and splits it into 60% training and 40% testing sets
df = loadtxt("drive/MyDrive/GE_practicum/diabetes.csv", delimiter=",", skiprows=1)

x_train = df[0:int(0.6*len(df)),0:8]
y_train = df[0:int(0.6*len(df)),8]

x_test = df[int(0.6*len(df)):,0:8]
y_test = df[int(0.6*len(df)):,8]

# **Quantization aware training**

In [13]:
# Loads the base model for ANN tests and gets baseline accuracy for result comparison
base_model = tf.keras.models.load_model('drive/MyDrive/GE_practicum/ANN_base')
base_model.fit(x_train,y_train,epochs = 50,validation_data = (x_test,y_test), batch_size=10)
base_model.summary()

Epoch 1/50
46/46 [==============================] - 1s 9ms/step - loss: 0.5690 - accuracy: 0.7109 - val_loss: 0.6278 - val_accuracy: 0.6916
Epoch 2/50
46/46 [==============================] - 0s 3ms/step - loss: 0.5128 - accuracy: 0.7630 - val_loss: 0.5717 - val_accuracy: 0.7532
Epoch 3/50
46/46 [==============================] - 0s 3ms/step - loss: 0.5050 - accuracy: 0.7457 - val_loss: 0.5518 - val_accuracy: 0.7305
Epoch 4/50
46/46 [==============================] - 0s 4ms/step - loss: 0.4927 - accuracy: 0.7500 - val_loss: 0.6145 - val_accuracy: 0.7175
Epoch 5/50
46/46 [==============================] - 0s 3ms/step - loss: 0.5054 - accuracy: 0.7696 - val_loss: 0.6198 - val_accuracy: 0.6558
Epoch 6/50
46/46 [==============================] - 0s 4ms/step - loss: 0.5038 - accuracy: 0.7457 - val_loss: 0.6741 - val_accuracy: 0.6558
Epoch 7/50
46/46 [==============================] - 0s 6ms/step - loss: 0.5067 - accuracy: 0.7630 - val_loss: 0.5922 - val_accuracy: 0.7305
Epoch 8/50
46/46 [==

In [14]:

quantize_model = tfmot.quantization.keras.quantize_model
model = quantize_model(base_model)

opt = tf.keras.optimizers.Adam(lr = 1e-3,decay = 1e-5)
model.compile(optimizer = opt,loss = "binary_crossentropy" , metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer_1 (QuantizeL  (None, 8)                3         
 ayer)                                                           
                                                                 
 quant_dense_8 (QuantizeWrap  (None, 128)              1157      
 perV2)                                                          
                                                                 
 quant_dense_9 (QuantizeWrap  (None, 64)               8261      
 perV2)                                                          
                                                                 
 quant_dense_10 (QuantizeWra  (None, 32)               2085      
 pperV2)                                                         
                                                                 
 quant_dense_11 (QuantizeWra  (None, 1)               

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [15]:
# Performs quantization aware training
model.fit(x_train,y_train,epochs = 50,validation_data = (x_test,y_test), batch_size=10)
model.save('test')

Epoch 1/50
46/46 [==============================] - 2s 15ms/step - loss: 0.4942 - accuracy: 0.7522 - val_loss: 0.6148 - val_accuracy: 0.7403
Epoch 2/50
46/46 [==============================] - 0s 6ms/step - loss: 0.4297 - accuracy: 0.8000 - val_loss: 0.6278 - val_accuracy: 0.6688
Epoch 3/50
46/46 [==============================] - 0s 6ms/step - loss: 0.4142 - accuracy: 0.8087 - val_loss: 0.6172 - val_accuracy: 0.7045
Epoch 4/50
46/46 [==============================] - 0s 6ms/step - loss: 0.4284 - accuracy: 0.7978 - val_loss: 0.6297 - val_accuracy: 0.7045
Epoch 5/50
46/46 [==============================] - 0s 8ms/step - loss: 0.4125 - accuracy: 0.8109 - val_loss: 0.6652 - val_accuracy: 0.6948
Epoch 6/50
46/46 [==============================] - 0s 9ms/step - loss: 0.4189 - accuracy: 0.7891 - val_loss: 0.6117 - val_accuracy: 0.7143
Epoch 7/50
46/46 [==============================] - 0s 8ms/step - loss: 0.4132 - accuracy: 0.7978 - val_loss: 0.6968 - val_accuracy: 0.6851
Epoch 8/50
46/46 [=

INFO:tensorflow:Assets written to: test/assets


INFO:tensorflow:Assets written to: test/assets


# **Quantization and conversion to tflite**

In [16]:
# Conversion to tflite

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [
tf.lite.OpsSet.TFLITE_BUILTINS,
tf.lite.OpsSet.SELECT_TF_OPS
]
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

open('ANN_quant.tflite', 'wb').write(tflite_model)

15736

In [17]:
# Check tflite model characteristics
tf.lite.experimental.Analyzer.analyze(model_content=tflite_model)

=== TFLite ModelAnalyzer ===

Your TFLite model has '1' subgraph(s). In the subgraph description below,
T# represents the Tensor numbers. For example, in Subgraph#0, the QUANTIZE op takes
tensor #0 as input and produces tensor #5 as output.

Subgraph#0 main(T#0) -> [T#15]
  Op#0 QUANTIZE(T#0) -> [T#5]
  Op#1 FULLY_CONNECTED(T#5, T#6, T#3) -> [T#7]
  Op#2 FULLY_CONNECTED(T#7, T#8, T#4) -> [T#9]
  Op#3 FULLY_CONNECTED(T#9, T#10, T#1) -> [T#11]
  Op#4 FULLY_CONNECTED(T#11, T#12, T#2) -> [T#13]
  Op#5 LOGISTIC(T#13) -> [T#14]
  Op#6 DEQUANTIZE(T#14) -> [T#15]

Tensors of Subgraph#0
  T#0(serving_default_dense_8_input:0) shape_signature:[-1, 8], type:FLOAT32
  T#1(dense_10/bias) shape:[32], type:INT32 RO 128 bytes
  T#2(dense_11/bias) shape:[1], type:INT32 RO 4 bytes
  T#3(dense_8/bias) shape:[128], type:INT32 RO 512 bytes
  T#4(dense_9/bias) shape:[64], type:INT32 RO 256 bytes
  T#5(sequential_2/quantize_layer_1/AllValuesQuantize/FakeQuantWithMinMaxVars;quantize_layer_1/quantize_layer_1_mi

In [18]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
input_details = interpreter.get_input_details()

interpreter.allocate_tensors()
output_details = interpreter.get_output_details()

#Predictions from TFLite model
tfl_pred = []
tfl_pred_class = []
for i in range(len(x_test)):
    interpreter.set_tensor(input_details[0]["index"], x_test.astype('float32')[i:i+1,:])
    interpreter.invoke()
    result = interpreter.get_tensor(output_details[0]["index"])
    tfl_pred.append(result)
    tfl_pred_class.append(int(result[0]*2))

right_pred = [y_test[i] == tfl_pred_class[i] for i in range(len(y_test))]
acc = sum(right_pred)/len(right_pred)
print(acc)

0.711038961038961


# **Conversion to C array**

In [19]:
# Function: Convert some hex value into an array for C programming
def hex_to_c_array(hex_data, var_name):

  c_str = ''

  # Create header guard
  c_str += '#ifndef ' + var_name.upper() + '_H\n'
  c_str += '#define ' + var_name.upper() + '_H\n\n'

  # Add array length at top of file
  c_str += '\nunsigned int ' + var_name + '_len = ' + str(len(hex_data)) + ';\n'

  # Declare C variable
  c_str += 'unsigned char ' + var_name + '[] = {'
  hex_array = []
  for i, val in enumerate(hex_data) :

    # Construct string from hex
    hex_str = format(val, '#04x')

    # Add formatting so each line stays within 80 characters
    if (i + 1) < len(hex_data):
      hex_str += ','
    if (i + 1) % 12 == 0:
      hex_str += '\n '
    hex_array.append(hex_str)

  # Add closing brace
  c_str += '\n ' + format(' '.join(hex_array)) + '\n};\n\n'

  # Close out header guard
  c_str += '#endif //' + var_name.upper() + '_H'

  return c_str

In [20]:
# Write TFLite model to a C source (or header) file
with open("ANN_quant" + '.h', 'w') as file:
  file.write(hex_to_c_array(tflite_model, "ANN_quant"))